## 1 factor Hull-White model
The dynamics of short rate $r(t)$ in Hull-hite model is given by

$
dr(t) = \left( \theta(t) - a r(t) \right) dt + \sigma dW(t),
$

where $\theta(t)$ is mean level, $a$ is mean revertion, $\sigma$ is volatility and $W(t)$ is winer process in probaility space(###).
This dynamics is easily solved and the solution is given by

\begin{align}
d(e^{at} r(t) ) &= a r e^{at}dt + e^{at}dr \\
&= e^{at} \theta(t) dt + e^{at} \sigma dW(t).
\end{align}

Thus, with partial integral from 0 to t, short rate is given by 
\begin{align}
r(t) = e^{-at} r(0) + \int_{0}^{t} e^{a(u-t)} \theta(u) dt + \int_{0}^{t} e^{a(u-t)} \sigma dW(u).
\end{align}

The dynamics of this stochastic process is followed by normal distoribution with  mean and variance given by
\begin{align}
& e^{-at} r(0) + \int_{0}^{t} e^{a(u-t)} \theta(u) dt, \\
& \frac{\sigma^{2}}{2a}\left( 1 - e^{-2at} \right).
\end{align}

In [1]:
import working_day.return_working_day as wd
import discount_curve.discount_curve as dc
import csv
import pandas as pd
import datetime

In [14]:
jpy_IR_obj = dc.discount_factor('DF_input/IR_data.csv', 'JPY', '2018/1/18')
jpy_IR_DF_list = jpy_IR_obj.get_DF_list()
pd.DataFrame(jpy_IR_DF_list, columns=['num', 'date', 'DF']).head()

,num,date,DF
0,0,2018/01/18,1.000000
1,1,2018/01/19,0.999961
2,2,2018/01/20,0.999921
3,3,2018/01/21,0.999881
4,4,2018/01/22,0.999841


In [37]:
class Hull_White:
    def __init__(self, mean_reversion, volatility, DF):
        self._mean_reversion = mean_reversion
        self._volatility = volatility
        self._DF = DF
        
    def get_DF

In [38]:
jpy_IR_obj = dc.discount_factor('DF_input/IR_data.csv', 'JPY', '2018/1/18')
jpy_IR_DF_list = jpy_IR_obj.get_DF_list()
hull_white_obj = Hull_White(0.1, 0.11, jpy_IR_DF_list)

In [39]:
jpy_IR_DF_list

[[0, '2018/01/18', 1.0],
 [1, '2018/01/19', 0.99996061798936],
 [2, '2018/01/20', 0.9999205802928088],
 [3, '2018/01/21', 0.9998805425962577],
 [4, '2018/01/22', 0.9998405048997066],
 [5, '2018/01/23', 0.9998004672031553],
 [6, '2018/01/24', 0.9997604295066042],
 [7, '2018/01/25', 0.999720391810053],
 [8, '2018/01/26', 0.9996803541135019],
 [9, '2018/01/27', 0.9996403164169507],
 [10, '2018/01/28', 0.9995968166255648],
 [11, '2018/01/29', 0.9995533168341787],
 [12, '2018/01/30', 0.9995098170427927],
 [13, '2018/01/31', 0.9994663172514067],
 [14, '2018/02/01', 0.9994228174600207],
 [15, '2018/02/02', 0.9993793176686347],
 [16, '2018/02/03', 0.9993358178772487],
 [17, '2018/02/04', 0.9992923180858627],
 [18, '2018/02/05', 0.9992488182944766],
 [19, '2018/02/06', 0.9992053185030907],
 [20, '2018/02/07', 0.9991618187117047],
 [21, '2018/02/08', 0.9991183189203187],
 [22, '2018/02/09', 0.9990748191289327],
 [23, '2018/02/10', 0.9990313193375467],
 [24, '2018/02/11', 0.9989878195461607],
 [2